In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('../input/time-series-datasets/Electric_Production.csv')

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:

df.rename(columns={'IPG2211A2N':'Consumption'}, inplace=True)
df.head()

In [ ]:
from matplotlib import pyplot
df.plot()
pyplot.show()

In [ ]:
df['Date'] = pd.to_datetime(df['DATE'])
train = df.set_index(['Date'], drop=True)
train.head()

In [ ]:
train.plot(style ='k.')

In [ ]:
import seaborn as sns
sns.boxplot(x=train['Consumption'])

In [ ]:
Naive Forecast 

In [ ]:
Benchmark_Model = pd.concat([train, train.shift(1)], axis=1)
Benchmark_Model.head()

In [ ]:
Benchmark_model = Benchmark_Model.drop(['DATE'],axis =1)
Benchmark_model.head()

In [ ]:
Benchmark_model.columns = ['Consumptions', 'Naive_forecast']
Benchmark_model.dropna(inplace=True)
Benchmark_model.head()

In [ ]:
Benchmark_model.plot()
pyplot.show()

In [ ]:
from sklearn.metrics import mean_squared_error
Benchmark_mean_error = mean_squared_error(Benchmark_model.Consumptions, Benchmark_model.Naive_forecast)
np.sqrt(Benchmark_mean_error)

In [ ]:
from sklearn.metrics import r2_score
Score = r2_score(Benchmark_model.Consumptions, Benchmark_model.Naive_forecast)
print(Score)

In [ ]:
Arima Forecast

In [ ]:
train.plot()
pyplot.show()

In [ ]:
from statsmodels.tsa.stattools import adfuller
Stationary_result=adfuller(train['Consumption'])

In [ ]:

print('ADF Statistic: %f' % Stationary_result[0])
print('p-value: %f' % Stationary_result[1])
print('Critical Values')
for key, value in Stationary_result[4].items():
	print('\t%s: %.3f' % (key, value))

In [ ]:
Seasonality Test 

In [ ]:
train['Seasonal_Adjusted'] = train['Consumption'] - train['Consumption'].shift(12)
train.dropna(inplace=True)

In [ ]:
import scipy.stats as stats
#Levene's test centered at the mean
stats.levene(train['Consumption'], train['Seasonal_Adjusted'], center='mean')

In [ ]:
from scipy.stats import f_oneway
f_oneway(train['Consumption'], train['Seasonal_Adjusted'])

In [ ]:
import matplotlib.pyplot as plt
from pandas.plotting import autocorrelation_plot
autocorrelation_plot(train['Consumption'])
plt.show()

In [ ]:
out = train['Consumption']
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
#plot_acf(Aut_acf ) 
plot_acf(out,lags=40)

In [ ]:
train.plot()
pyplot.show()

In [ ]:
!pip install pmdarima

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
import statsmodels.api as sm

In [ ]:

fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(train['Seasonal_Adjusted'].iloc[13:],lags=40,ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(train['Seasonal_Adjusted'].iloc[13:],lags=40,ax=ax2)

In [ ]:
from statsmodels.tsa.arima_model import ARMA
model = ARMA(train['Seasonal_Adjusted'],order=(4,0,2))
mode_fit = model.fit() 
mode_fit.summary()

In [ ]:
train['forecast']=mode_fit.predict(start=300,end=384,dynamic=True)
train[['Consumption','forecast']].plot(figsize=(10,6))

In [ ]:
train.dropna(inplace=True)
train['forecast']=mode_fit.predict(start=300,end=384,dynamic=True)
train[['Consumption','forecast']].plot(figsize=(10,6))

In [ ]:
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(train['Consumption'], train['forecast']))

In [ ]:
Arma_Score = r2_score(train['Consumption'],train['forecast'])
print(Arma_Score)

In [ ]:
AUTOMATIC ARIMA MODEL 

In [ ]:
df['Date'] = pd.to_datetime(df['DATE'])
train = df.set_index(['Date'], drop=True)
train['Seasonal_Adjusted'] = train['Consumption'] - train['Consumption'].shift(12)
train.dropna(inplace=True)


In [ ]:
from pmdarima.arima import auto_arima
Ist_diff_Auto_model = auto_arima(train['Seasonal_Adjusted'], start_p=0, start_q=0, max_p=5, max_q=5, seasonal=True, d=2, D=0, 
                            trace=True, suppress_warnings=True, error_action='ignore', stepwise=True)
print(Ist_diff_Auto_model.summary())

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
model = ARIMA(train['Seasonal_Adjusted'],order=(5,2,0))
model_fit = model.fit() 
model_fit.summary()

In [ ]:
#train.dropna(inplace=True)
train['forecast']=model_fit.predict(start=300,end=384,dynamic=True)
train[['Consumption','forecast']].plot(figsize=(10,6))

In [ ]:
train.dropna(inplace=True)
np.sqrt(mean_squared_error(train['Consumption'], train['forecast']))

In [ ]:
Non_Arma_Score = r2_score(train['Consumption'], train['forecast'])
print(Non_Arma_Score)

In [ ]:
Non- seasonal Arima

In [ ]:
df['Date'] = pd.to_datetime(df['DATE'])
train = df.set_index(['Date'], drop=True)
train.dropna(inplace=True)


In [ ]:
model=sm.tsa.statespace.SARIMAX(train['Consumption'],order=(4,0,2),seasonal_order=(1,1,1,12))
results=model.fit()
results.summary()

In [ ]:
train['forecast']= results.predict(start=300,end=397,dynamic=True)
train[['Consumption','forecast']].plot(figsize=(10,6))

In [ ]:
results.plot_diagnostics(figsize=(9,7))
plt.show()

Top left: The residual errors seem to fluctuate around a mean of zero and have a uniform variance.

Top Right: The density plot suggest normal distribution with mean zero.

Bottom left: All the dots should fall perfectly in line with the red line. Any significant deviations would imply the distribution is skewed.

Bottom Right: The Correlogram, aka, ACF plot shows the residual errors are not autocorrelated. Any autocorrelation would imply that there is some pattern in the residual errors which are not explained in the model. So you will need to look for more X’s (predictors) to the model.

Overall, it seems to be a good fit. Let’s forecast.

In [ ]:
Future forecast 

In [ ]:
from pandas.tseries.offsets import DateOffset
future_dates=[train.index[-1]+ DateOffset(months=x)for x in range(0,24)]

future_datest_data=pd.DataFrame(index=future_dates[1:],columns=train.columns)

In [ ]:

future_data=pd.concat([train,future_datest_data])
future_data['forecast'] = results.predict(start = 397, end = 494, dynamic= True)  
future_data[['Consumption', 'forecast']].plot(figsize=(8, 4))

In [ ]:
train.dropna(inplace=True)
np.sqrt(mean_squared_error(train['Consumption'], train['forecast']))


In [ ]:
Non_Arma_Score = r2_score(train['Consumption'], train['forecast'])
print(Non_Arma_Score)